# Demo of PyPantograph

This example showcases **motivated proofs** in Lean. Consider a simple analysis proof that starts with "Let $\epsilon > 0$. Set $\delta := \min(\epsilon/2, \epsilon^2)$. The reader is left wondering how was the $\delta$ generated. In a motivated proof, $\delta$ would be left blank and then filled in when all of its constraints are in order.

## Setup

1. Clone the [PyPantograph](https://github.com/stanford-centaur/PyPantograph) repository.
2. Clone Mathlib4 [Mathlib4](https://github.com/leanprover-community/mathlib4) and checkout tag `v4.18.0`.
3. Run `lake exe cache get` in the directory of Mathlib4.

In [1]:
from pantograph.server import Server

# Modify the `project_path` here!
server = await Server.create(imports=["Mathlib"], project_path="/Users/aniva/Projects/contrib/formalization/mathlib4")

In [3]:
sketch = """
def convergesTo (s : ℕ → ℝ) (a : ℝ) :=
  ∀ ε > 0, ∃ N, ∀ n ≥ N, |s n - a| < ε

example (a : ℝ) : convergesTo (λ _ => a) a := sorry
"""
unit0, unit1 = await server.load_sorry_async(sketch)
unit1

CompilationUnit(i_begin=98, i_end=153, messages=["<anonymous>:5:0: warning: declaration uses 'sorry'\n"], invocations=None, goal_state=GoalState(state_id=1, goals=[Goal(variables=[Variable(t='ℝ', v=None, name='a')], target='convergesTo (fun x => a) a', sibling_dep=[], name=None, is_conversion=False)], _sentinel=[]), goal_src_boundaries=[[147, 152]], new_constants=None)

## A Presentation View Proof

We first exhibit a presentation view (non-motivated) proof.

In [18]:
state0 = unit1.goal_state
state1 = await server.goal_tactic_async(state0, 0, "intro ε _")
state2 = await server.goal_tactic_async(state1, 0, "use 0")
print(state2)

a : ℝ
ε : ℝ
a✝ : ε > 0
⊢ ∀ n ≥ 0, |(fun x => a) n - a| < ε


In [19]:
state3 = await server.goal_tactic_async(state2, 0, "intro n hn")
state4 = await server.goal_tactic_async(state3, 0, "rw [sub_self, abs_zero]")
state5 = await server.goal_tactic_async(state4, 0, "apply ‹ε > 0›")
state5.is_solved

True

## A Search View Proof

In [17]:
state0 = unit1.goal_state
state1 = await server.goal_tactic_async(state0, 0, "intro ε _")
state2 = await server.goal_tactic_async(state1, 0, "constructor")
print(state2)

a : ℝ
ε : ℝ
a✝ : ε > 0
⊢ ∀ n ≥ ?w, |(fun x => a) n - a| < ε
a : ℝ
ε : ℝ
a✝ : ε > 0
⊢ ℕ


In [11]:
print(state2)

a : ℝ
ε : ℝ
a✝ : ε > 0
⊢ ∀ n ≥ ?w, |(fun x => a) n - a| < ε
a : ℝ
ε : ℝ
a✝ : ε > 0
⊢ ℕ


In [13]:
state3 = await server.goal_tactic_async(state2, 0, "intro n hn")
state4 = await server.goal_tactic_async(state3, 0, "rw [sub_self, abs_zero]")
print(state4)

a : ℝ
ε : ℝ
a✝ : ε > 0
n : ℕ
hn : n ≥ ?w
⊢ 0 < ε
a : ℝ
ε : ℝ
a✝ : ε > 0
⊢ ℕ


In [15]:
state5 = await server.goal_tactic_async(state4, 0, "exact ‹ε > 0›")
print(state5)

a : ℝ
ε : ℝ
a✝ : ε > 0
⊢ ℕ


In [16]:
state6 = await server.goal_tactic_async(state5, 0, "exact 123")
print(state6)